In [137]:
# Importação das bibiliotecas
import pandas as pd
import lightgbm as lgb
from lightgbm import early_stopping, log_evaluation
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import accuracy_score, classification_report


In [138]:
#Dados tratados do IBOVESPA
dados_ibovespa = pd.read_excel("Dados Tratados - Ibovespa.xlsx")

In [139]:
dados_ibovespa.columns = dados_ibovespa.columns.str.strip().str.replace(' ', '_') # Remove espaços nas extremidades das colunas e substitui espaços internos por '_'

In [140]:
def converter_volume_str_to_num(valor): # Converte 'M' (Milhões) e 'B' (Bilhões) para a quantidade real de casas decimais
    valor = valor.replace('.', '').replace(',', '.')
    if 'M' in valor:
        return float(valor.replace('M', '')) * 1e6
    elif 'B' in valor:
        return float(valor.replace('B', '')) * 1e9
    else:
        return float(valor)

dados_ibovespa['Vol.'] = dados_ibovespa['Vol.'].apply(converter_volume_str_to_num) # Aplica função na coluna 'Vol.'

In [141]:
dados_ibovespa['Data'] = pd.to_datetime(dados_ibovespa['Data'], dayfirst=True) # Transforma em datatype "datetime"
dados_ibovespa = dados_ibovespa.sort_values('Data') # Organiza os registros da coluna 'Data' começando do mais antigo

In [142]:
dados_ibovespa['Target'] = dados_ibovespa['Variacao'].shift(-1) # Cria uma coluna denominada 'Target', pega os valores da coluna 'Variacao' e desloca uma linha para cima, assim pegando o valor do dia atual para prever o do dia seguinte
dados_ibovespa = dados_ibovespa.dropna() # Remoção de registros nulos
dados_ibovespa['Target'] = dados_ibovespa['Target'].astype(int) # Transforma o datatype da coluna Target em int

In [143]:
# Fórmulas
dados_ibovespa['retorno_diario'] = (dados_ibovespa['Ultimo'] - dados_ibovespa['Abertura']) / dados_ibovespa['Abertura'] # Variação diária do valor negociado do índice IBOVESPA
dados_ibovespa['volatilidade'] = (dados_ibovespa['Maxima'] - dados_ibovespa['Minima']) / dados_ibovespa['Abertura'] # Variação relativa diária do índice Ibovespa em relação ao valor de abertura
dados_ibovespa['posicao_no_dia'] = (dados_ibovespa['Ultimo'] - dados_ibovespa['Minima']) / (dados_ibovespa['Maxima'] - dados_ibovespa['Minima'] + 1e-6) # Relação do fechamento diário (próximo de 1 = perto da máxima; próximo de 0 = perto da mínima)
# 1e-6 evita divisão por zero, que pode acontecer em alguns casos
dados_ibovespa['media_ultimos_3_dias'] = dados_ibovespa['Ultimo'].rolling(3).mean() # Média móvel dos últimos 3 dias do valor de fechamento
dados_ibovespa['media_ultimos_5_dias'] = dados_ibovespa['Ultimo'].rolling(window=5).mean()   # Média móvel dos últimos 5 dias do valor de fechamento
dados_ibovespa['media_ultimos_10_dias'] = dados_ibovespa['Ultimo'].rolling(window=10).mean() # Média móvel dos últimos 10 dias do valor de fechamento
dados_ibovespa['media_exp_5_dias'] = dados_ibovespa['Ultimo'].ewm(span=5, adjust=False).mean()   # Média exponencial dos últimos 5 dias do valor de fechamento
dados_ibovespa['media_exp_10_dias'] = dados_ibovespa['Ultimo'].ewm(span=10, adjust=False).mean() # Média exponencial dos últimos 10 dias do valor de fechamento

dados_ibovespa['retorno_3_dias'] = dados_ibovespa['Ultimo'].pct_change(3) # Retorno percentual dos últimos 3 dias

In [144]:
dados_ibovespa['Dia_da_Semana'] = dados_ibovespa['Dia_da_Semana'].astype('category').cat.codes # Substitui string dos dias da semana por índices numéricos

In [145]:
dados_ibovespa = dados_ibovespa.dropna() # Remoção de registros nulos (primeiros registros das fórmulas que calculam média baseando-se nos últimos)
features = dados_ibovespa.drop(columns=['Data', 'Variacao', 'Target']) #Dataframe com as features
alvo = dados_ibovespa['Target']

In [146]:
numero_folds = TimeSeriesSplit(n_splits = 3) # Definir quantidade de folds --------------------------------------------------------------------------
accuracies = []
reports = []

In [155]:

fold = 1
for train_index, val_index in numero_folds.split(features): # Aumenta progressivamente a base para treino do modelo
    print(f"\nFold {fold}")
    features_train, features_val = features.iloc[train_index], features.iloc[val_index] # Variáveis criadas
    alvo_train, alvo_val = alvo.iloc[train_index], alvo.iloc[val_index] # O que o modelo deve prever

    model = lgb.LGBMClassifier( # Parâmetros do modelo (treinamento)
        n_estimators = 2000, # número de árvores
        learning_rate = 0.01, # Tempo para o aprendizado
        num_leaves = 60, # Quantidade de "folhas" das árvores
        max_depth = -1, #
        class_weight = 'balanced',
        random_state = 50, # Início aleatório do algoritmo
        verbosity = -1 # Remove mensagens de log
    )

    model.fit(
    features_train, alvo_train,
    eval_set=[(features_val, alvo_val)],
    eval_metric='binary_logloss', # Mede a qualidade das previsões do modelo (abaixo de 0.5 = considerado bom, perto de 0.5 = deve melhorar, muito acima de 0.5 = ruim)
    callbacks=[
        early_stopping(stopping_rounds=100),
        log_evaluation(period=100)
    ]
)

    alvo_pred = model.predict(features_val)
    acc = accuracy_score(alvo_val, alvo_pred)
    print(f"Acurácia Fold {fold}: {acc:.5f}")

    accuracies.append(acc)
    reports.append(classification_report(alvo_val, alvo_pred, output_dict=True))

    fold += 1

    modelo_teste = lgb.LGBMClassifier( # Parâmetros do modelo (teste)
   n_estimators = 2000,
   learning_rate = 0.01,
   num_leaves = 60,
   max_depth = -1,
   class_weight = 'balanced',
   random_state = 50,
   verbosity = -1
)

modelo_teste.fit(features, alvo); # Teste com todos os dados


Fold 1
Training until validation scores don't improve for 100 rounds
[100]	valid_0's binary_logloss: 0.562695
[200]	valid_0's binary_logloss: 0.575711
Early stopping, best iteration is:
[112]	valid_0's binary_logloss: 0.561309
Acurácia Fold 1: 0.72403

Fold 2
Training until validation scores don't improve for 100 rounds
[100]	valid_0's binary_logloss: 0.563267
[200]	valid_0's binary_logloss: 0.540719
[300]	valid_0's binary_logloss: 0.534734
Early stopping, best iteration is:
[277]	valid_0's binary_logloss: 0.533726
Acurácia Fold 2: 0.75649

Fold 3
Training until validation scores don't improve for 100 rounds
[100]	valid_0's binary_logloss: 0.552247
[200]	valid_0's binary_logloss: 0.536027
Early stopping, best iteration is:
[183]	valid_0's binary_logloss: 0.534615
Acurácia Fold 3: 0.75000


In [150]:
dia_seguinte = features.tail(1) # Pega último registro e o utiliza pro algoritmo prever a subida ou queda do IBOVESPA do dia seguinte
previsao = modelo_teste.predict(dia_seguinte)
print("\nIBOVESPA irá subir amanhã?", "Sim" if previsao[0] == 1 else "Não")


IBOVESPA irá subir amanhã? Não
